# Bibliotecas

In [ ]:
%matplotlib inline
import os
import cv2
import csv
import glob
import pandas as pd
import numpy as np
import random
import itertools
from collections import Counter
from math import ceil
import matplotlib.pyplot as plt

#Descobrir autilidade
from tqdm.notebook import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# Funções úteis

In [ ]:
def skip_csv_header(file):
    has_header = csv.Sniffer().has_header(file.read(1024))
    file.seek(0)
    if has_header:
        next(file)


def total_image_list(image_folder_path):
    total_img_list = [os.path.basename(img_path_name) for img_path_name in glob.glob(os.path.join(image_folder_path, "*.jpg"))]
    return total_img_list

def draw_rect(img, bboxes, color=None):
    img = img.copy()
    bboxes = bboxes[:, :4]
    bboxes = bboxes.reshape(-1, 4)
    for bbox in bboxes:
        pt1, pt2 = (bbox[0], bbox[1]), (bbox[2], bbox[3])
        pt1 = int(pt1[0]), int(pt1[1])
        pt2 = int(pt2[0]), int(pt2[1])
        img = cv2.rectangle(img.copy(), pt1, pt2, color, int(max(img.shape[:2]) / 200))
    return img

def plot_multiple_img(img_matrix_list, title_list, ncols, main_title=""):
    fig, myaxes = plt.subplots(figsize=(20, 15), nrows=ceil(len(img_matrix_list) / ncols), ncols=ncols, squeeze=False)
    fig.suptitle(main_title, fontsize = 30)
    fig.subplots_adjust(wspace=0.3)
    fig.subplots_adjust(hspace=0.3)
    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        myaxes[i // ncols][i % ncols].imshow(img)
        myaxes[i // ncols][i % ncols].set_title(title, fontsize=15)
    plt.show()

Neste kernel, apresento algumas funções utilitárias para fazer uma verificação de sanidade em imagens, bem como algumas funções que você pode reutilizar para projetos futuros quando quiser plotar várias imagens em uma grade. Uma prévia de como um gráfico de caixa delimitadora múltipla é assim:

![](https://i.ibb.co/9GXMpWT/img.png)

# 1.Ler e carregar o conjunto de dados

In [ ]:
train = pd.read_csv("/kaggle/input/global-wheat-detection/train.csv")  
image_folder_path = "/kaggle/input/global-wheat-detection/train/"

## 1.1. Bounding boxes

In [ ]:
bboxes = np.stack(train['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x_min', 'y_min', 'width', 'height']):
    train[column] = bboxes[:,i]
    
train["x_max"] = train.apply(lambda col: col.x_min + col.width, axis=1)
train["y_max"] = train.apply(lambda col: col.y_min + col.height, axis = 1)
train.drop(columns=['bbox'], inplace=True)

In [ ]:
# Obeservar dados 
train.head()

## 1.2. Verificação de alcance nas coordenadas das bounding boxes

Verificação de intervalo na coordenada da caixa delimitadora Além disso, devido aos problemas internos de flutuação do python, pode haver valores estranhos como valores negativos ou que somam mais de 1024 em x_max, y_max. Precisamos ter cuidado aqui.

Este é um problema sério que pode ocorrer quando você normaliza a caixa delimitadora, pode exceder 1 e isso causará um erro, especialmente se você decidir aumentar as imagens também.

In [ ]:
train[train["x_max"] > 1024]
train[train["y_max"] > 1024]
train[train["x_min"] < 0]
train[train["y_min"] < 0]

A única razão pela qual, por exemplo, a linha 31785 tem x_max mais de 1024, é devido à rotulagem do conjunto de dados original. Vejamos as respectivas linhas problemáticas. Por exemplo, na linha 31785, o x_min fornecido é 873,200012, e quando você adiciona isso à largura de 150,800003, fornece 1024,000015, que já excede o tamanho da imagem. Então você tem que arredondar para baixo. E, tanto quanto eu sinto, as caixas delimitadoras, quando desnormalizadas, devem ser em números inteiros. Mas esta é apenas a minha opinião. Vamos mudar esses valores problemáticos para 1024

In [ ]:
x_max = np.array(train["x_max"].values.tolist())
y_max = np.array(train["y_max"].values.tolist())
train["x_max"] = np.where(x_max > 1024, 1024, x_max).tolist()
train["y_max"] = np.where(y_max > 1024, 1024, y_max).tolist()

Podemos deletar colunas de largura e altura porque não precisamos delas, pode ser facilmente retirado das próprias imagens.

In [ ]:
del train["width"]
del train["height"]
del train["source"]
train.head()

In [ ]:
train["class"] = "1"
train.head()

# 2. Verifique se as extensões de imagem são todas jpg

Primeiro, verificamos se todas as imagens da pasta train estão no formato .jpg. É melhor verificar porque se houver uma mistura de tipos de imagem, podemos enfrentar problemas mais tarde.

In [ ]:
def check_file_type(image_folder_path):
    extension_type = []
    file_list = os.listdir(image_folder_path)
    
    for file in file_list:
        extension_type.append(file.rsplit(".", 1)[1].lower())
    print(Counter(extension_type).keys())
    print(Counter(extension_type).values())
    
check_file_type(image_folder_path)

Bom, parece que todas as nossas imagens na pasta estão no formato .jpg. Em seguida, é melhor anexar .jpg atrás de todo o image_id no dataframe. Isso nos tornará mais fácil manipular os dados posteriormente.

In [ ]:
train["image_id"] = train["image_id"].apply(lambda x: str(x) + ".jpg")
train.head()

In [ ]:
train["image_id"] = train["image_id"].astype("str")
train.to_csv("wheat.csv", index=False)

# 3. Verifique se há imagens corrompidas e se todas as imagens são 1.024 por 1.024

In [ ]:
def check_image_size(image_folder_path):
    total_img_list = glob.glob(os.path.join(image_folder_path,"*"))
    counter = 0
    for image in tqdm(total_img_list, desc = "Checking in progress"):
        try:
            img = cv2.imread(image)
            height, width = img.shape[1], img.shape[0]
            if not (height == 1024 and width == 1024):
                counter = counter + 1
        except:
            print("This {} is problematic.".format(image))
    return counter 

In [ ]:
check_image_size(image_folder_path)

Ótimo, na verdade todas as nossas imagens têm tamanho de 1024 x 1024. E o bom é que esse código também nos ajuda a verificar se há imagens corrompidas, portanto, se houver uma imagem corrompida, com certeza irá mostrar que o contador é diferente de zero. E a partir daí você pode verificar qual imagem está causando o problema.

# 4. Verificação de sanidade entre o csv do trem e as imagens do treino 
We will write a function to check if the number of unique image_ids match the number of unique images in the folder.

In [ ]:
wheat = pd.read_csv("wheat.csv") 
image_folder_path = "/kaggle/input/global-wheat-detection/train/"
image_annotation_file = "wheat.csv"
wheat.head()

In [ ]:
def sanity_tally(image_folder_path, image_annotation_file):
    img_dict = {}
    with open(image_annotation_file, "r") as file:
        skip_csv_header(file)
        for row in file:
            try:
                image_name, x_min, y_min, x_max, y_max, class_idx = row.split(",")
                if image_name not in img_dict:
                    img_dict[image_name] = list()
                img_dict[image_name].append(
                    [float(x_min), float(y_min), float(x_max), float(y_max), int(class_idx)]
                )
            except ValueError:
                print("Could not convert float to string, likely that your data has empty values.")
        
    img_annotation_list = [*img_dict]
    total_img_list = total_image_list(image_folder_path)
    if set(img_annotation_list) == set(total_img_list):
        print("Sanity Check Status: True")
    else:
        print("Sanity Check Status: Failed. \nThe elements in wheat/train.csv but not in the train image folder is {}. \nThe elements in train image folder but not in wheat/train.csv is {}".format(
                set(img_annotation_list) - set(total_img_list), set(total_img_list) - set(img_annotation_list)))
        return list(set(img_annotation_list) - set(total_img_list)), list(set(total_img_list) - set(img_annotation_list))

In [ ]:
set_diff1, set_diff2 = sanity_tally(image_folder_path, image_annotation_file = image_annotation_file)

print("There are {} images without annotations in the train/wheat.csv".format(len(set_diff2)))

Como podemos ver acima, existem 49 imagens sem anotações de caixa delimitadora porque elas não têm trigos na imagem e, portanto, não aparecem no train.csv. Pode ser uma ideia colocar essas 49 imagens dentro do train.csv e rotulá-las como 0.

# 5. Plotando várias imagens
Here we define a nice function that is useful not only for this competition, but for similar project as well. Note that we used our utility function here to plot them. One can tune the parameters accordingly.

In [ ]:
def plot_random_images(image_folder_path, image_annotation_file, num = 12):
    img_dict = {}
    with open(image_annotation_file, "r") as file:
        skip_csv_header(file)
        for row in file:
            try:
                image_name, x_min, y_min, x_max, y_max, class_idx = row.split(",")
                if image_name not in img_dict:
                    img_dict[image_name] = list()
                img_dict[image_name].append(
                    [float(x_min), float(y_min), float(x_max), float(y_max), int(class_idx)]
                )
            except ValueError:
                print("Could not convert float to string, likely that your data has empty values.")

    # randomly choose 12 images to plot
    img_files_list = np.random.choice(list(img_dict.keys()), num)
    print("The images' names are {}".format(img_files_list))
    img_matrix_list = []
    
    for img_file in img_files_list:
        image_file_path = os.path.join(image_folder_path, img_file)
        img = cv2.imread(image_file_path)[:,:,::-1]  
        img_matrix_list.append(img)

    
    return plot_multiple_img(img_matrix_list, title_list = img_files_list, ncols = 4, main_title="Wheat Images")

Aqui vemos uma bela grade de 12 imagens plotadas.

In [ ]:
plot_random_images(image_folder_path, image_annotation_file, num = 12)

# 6. Plotagem de várias imagens com caixas delimitadoras

Na detecção de objetos com caixas delimitadoras, é sempre uma boa ideia plotar aleatoriamente algumas imagens com suas caixas delimitadoras para verificar as coordenadas incorretas da caixa delimitadora. Embora eu deva dizer que nesta competição em particular, há muitas imagens com muitas caixas delimitadoras e, portanto, você deve examinar claramente.

In [ ]:
def random_bbox_check(image_folder_path, image_annotation_file, num = 12):
    img_dict = {}
    labels = ["wheat", "no wheat"]
    with open(image_annotation_file, "r") as file:
        skip_csv_header(file)
        for row in file:
            try:
                image_name, x_min, y_min, x_max, y_max, class_idx = row.split(",")
                if image_name not in img_dict:
                    img_dict[image_name] = list()
                img_dict[image_name].append(
                    [float(x_min), float(y_min), float(x_max), float(y_max), int(class_idx)]
                )
            except ValueError:
                print("Could not convert float to string, likely that your data has empty values.")

    # randomly choose 12 image.
    img_files_list = np.random.choice(list(img_dict.keys()), num)
    print("The images' names are {}".format(img_files_list))
    image_file_path_list = []

    bbox_list = []
    img_matrix_list = []
    random_image_matrix_list = []
    
    for img_file in img_files_list:
        image_file_path = os.path.join(image_folder_path, img_file)
        img = cv2.imread(image_file_path)[:,:,::-1]  
        height, width, channels = img.shape
        bbox_list.append(img_dict[img_file])
        img_matrix_list.append(img)

    
    final_bbox_list = []
    for bboxes, img in zip(bbox_list, img_matrix_list):
        final_bbox_array = np.array([])
        #bboxes is a 2d array [[...], [...]]
        for bbox in bboxes:
            bbox = np.array(bbox).reshape(1,5)
            final_bbox_array = np.append(final_bbox_array, bbox)
        final_bbox_array = final_bbox_array.reshape(-1,5)
        random_image = draw_rect(img.copy(), final_bbox_array.copy(), color = (255,0,0))
        random_image_matrix_list.append(random_image)
    plot_multiple_img(random_image_matrix_list, title_list = img_files_list, ncols = 4, main_title="Bounding Box Wheat Images")

In [ ]:
#Aplicar
random_bbox_check(image_folder_path, image_annotation_file)

# 7. Aumentos

O aumento é uma técnica importante para aumentar artificialmente o tamanho dos dados. Em particular, quando o conjunto de dados é pequeno, o aumento antes do treinamento do modelo ajudará a rede a aprender melhor.

In [ ]:
# Albumentations
import albumentations as A

In [ ]:
image_folder_path = "/kaggle/input/global-wheat-detection/train/"
chosen_image = cv2.imread(os.path.join(image_folder_path, "1ee6b9669.jpg"))[:,:,::-1]
plt.imshow(chosen_image)

In [ ]:
chosen_image_dataframe = wheat.loc[wheat["image_id"]=="1ee6b9669.jpg",["x_min","y_min","x_max","y_max","class"]]
bbox_array_of_chosen_image = np.array(chosen_image_dataframe.values.tolist())
bbox_array_of_chosen_image.shape

In [ ]:
draw_chosen_image = draw_rect(chosen_image.copy(), bbox_array_of_chosen_image.copy(), color = (255,0,0))
plt.imshow(draw_chosen_image)

In [ ]:
albumentation_list = [A.RandomSunFlare(p=1), A.RandomFog(p=1), A.RandomBrightness(p=1),
                      A.RandomCrop(p=1,height = 512, width = 512), A.Rotate(p=1, limit=90),
                      A.RGBShift(p=1), A.RandomSnow(p=1),
                      A.HorizontalFlip(p=1), A.VerticalFlip(p=1), A.RandomContrast(limit = 0.5,p = 1),
                      A.HueSaturationValue(p=1,hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=50)]

img_matrix_list = []
bboxes_list = []
for aug_type in albumentation_list:
    img = aug_type(image = chosen_image)['image']
    img_matrix_list.append(img)

img_matrix_list.insert(0,chosen_image)    

titles_list = ["Original","RandomSunFlare","RandomFog","RandomBrightness",
               "RandomCrop","Rotate", "RGBShift", "RandomSnow","HorizontalFlip", "VerticalFlip", "RandomContrast","HSV"]

##reminder of helper function
def plot_multiple_img(img_matrix_list, title_list, ncols, main_title=""):
    fig, myaxes = plt.subplots(figsize=(20, 15), nrows=3, ncols=ncols, squeeze=False)
    fig.suptitle(main_title, fontsize = 30)
    fig.subplots_adjust(wspace=0.3)
    fig.subplots_adjust(hspace=0.3)
    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        myaxes[i // ncols][i % ncols].imshow(img)
        myaxes[i // ncols][i % ncols].set_title(title, fontsize=15)
    plt.show()
    
plot_multiple_img(img_matrix_list, titles_list, ncols = 4,main_title="Different Types of Augmentations")

# 8. Caixas delimitadoras com albumentações

Lembre-se de que estamos usando nossa imagem escolhida como exemplo, por conveniência, vou lembrá-lo da matriz de imagem de imagens escolhidas e suas coordenadas de caixas delimitadoras abaixo. Mas há uma advertência aqui, minha matriz de caixas delimitadoras tem a forma [N, 5], onde o último elemento são os rótulos. Mas quando você quiser usar Albumentations para plotar caixas delimitadoras, use bboxes no formato de pascal_voc que é [x_min, y_min, x_max, y_max]; ele também leva em label_fields que são os rótulos de cada caixa delimitadora. Portanto, ainda precisamos fazer um pré-processamento simples abaixo.

In [ ]:
chosen_image = cv2.imread(os.path.join(image_folder_path, "1ee6b9669.jpg"))[:,:,::-1]
chosen_image_dataframe = wheat.loc[wheat["image_id"]=="1ee6b9669.jpg",["x_min","y_min","x_max","y_max"]]
bbox_array_of_chosen_image = np.array(chosen_image_dataframe.values.tolist())
labels_of_chosen_image = np.ones((len(bbox_array_of_chosen_image),))

In [ ]:
def draw_rect_with_labels(img, bboxes,class_id, class_dict, color=None):
    img = img.copy()
    bboxes = bboxes[:, :4]
    bboxes = bboxes.reshape(-1, 4)
    for bbox, label in zip(bboxes, class_id):
        pt1, pt2 = (bbox[0], bbox[1]), (bbox[2], bbox[3])
        pt1 = int(pt1[0]), int(pt1[1])
        pt2 = int(pt2[0]), int(pt2[1])
        class_name = class_dict[label]
        ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1) 
        img = cv2.rectangle(img.copy(), pt1, pt2, color, int(max(img.shape[:2]) / 200))
        img = cv2.putText(img.copy(), class_name, (int(bbox[0]), int(bbox[1]) - int(0.3 * text_height)), cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color = (255,255,255), lineType=cv2.LINE_AA)
    return img

In [ ]:
ver_flip = A.Compose([
        A.VerticalFlip(p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


ver_flip_annotations = ver_flip(image=chosen_image, bboxes=bbox_array_of_chosen_image, labels=labels_of_chosen_image)
ver_flip_annotations['bboxes'] = [list(bbox) for bbox in ver_flip_annotations['bboxes']]

In [ ]:
ver_flip = A.Compose([
        A.VerticalFlip(p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


ver_flip_annotations = ver_flip(image=chosen_image, bboxes=bbox_array_of_chosen_image, labels=labels_of_chosen_image)
ver_flip_annotations['bboxes'] = [list(bbox) for bbox in ver_flip_annotations['bboxes']]

In [ ]:
hor_flip = A.Compose([
        A.HorizontalFlip(p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


hor_flip_annotations = hor_flip(image=chosen_image, bboxes=bbox_array_of_chosen_image, labels=labels_of_chosen_image)
hor_flip_annotations['bboxes'] = [list(bbox) for bbox in hor_flip_annotations['bboxes']]


hor_flip_img = draw_rect_with_labels(img = hor_flip_annotations['image'], bboxes = np.array(hor_flip_annotations['bboxes']),
                          class_id = hor_flip_annotations['labels'], class_dict = {0: "background",1: "wheat"}, color=(255,0,0))
    
img_matrix_list = [draw_chosen_image, hor_flip_img]
titles_list = ["Original", "HorizontalFlipped"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 2,main_title="Horizontal Flip")

In [ ]:
transform = A.Compose([
    A.CoarseDropout(max_height=100, max_width=100, p = 1),
    A.RandomBrightnessContrast(p=0.9),
    A.HueSaturationValue(
                        hue_shift_limit=0.2,
                        sat_shift_limit=0.2,
                        val_shift_limit=0.2,
                        p=0.9,
                        )
])
chosen_image = cv2.imread(os.path.join(image_folder_path, "1ee6b9669.jpg"))[:,:,::-1]
augmented_image = transform(image=chosen_image)['image']
plt.imshow(augmented_image)